# **📔 TensorPrime**

#### ↖️ Click the ▶️ button after deciding on the options below.
#### 🔌 Make sure the TPU is enabled under *Runtime→Change runtime type*
#### 💡 Keep each notebook **open** in the browser to prevent disconnection. 📌 [the tab](https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open) or move them to a dedicated window for easy access.
#### ℹ️ This notebook uses our [TensorPrime](https://github.com/TPU-Mersenne-Prime-Search/TensorPrime) and our [PrimeNet Python script](https://github.com/tdulcet/Distributed-Computing-Scripts#primenet).
#### 📜 Please see the [wiki](TPU-Mersenne-Prime-Search/TensorPrime) for more information.
#### 🤷 Optionally, create a GIMPS/PrimeNet account [here](https://www.mersenne.org/update/) and [join](https://www.mersenne.org/jteam/) the “Portland State University” team!


In [ ]:
import os

#{ vertical-output: true, form-width: "20%", display-mode: "form" }

branch = 'master' #@param ['master'] {allow-input: true}
prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
TPU_type_of_work = '150 - First time PRP tests' #@param ['150 - First time PRP tests', '151 - Double-check PRP tests']
computer_number = 'Default (1)' #@param ['Default (1)', '2', '3', '4'] {allow-input: true}
output_type = 'TPU (TensorPrime)' #@param ['TPU (TensorPrime)']
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'TPU (TensorPrime)']


#@markdown #### 🐛 The *debug* option outputs TPU (TensorPrime) progress and status, then exits.


'''Load & cd into gdrive for persistent data'''
global path_dir
if not os.path.exists('/content/gdrive/MyDrive/'): # create your own notebook with our code
  print("Google Drive is not mounted at /content/gdrive. You will be prompted to sign in to Google to mount your GDrive.")
  from google.colab import drive
  drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive'


if not os.path.exists('/content/gdrive/MyDrive/GIMPS'):
  print("Creating GIMPS folder...")
  %mkdir -p GIMPS
%cd 'GIMPS'

!git clone https://github.com/TPU-Mersenne-Prime-Search/TensorPrime/ TensorPrime
%cd TensorPrime
!git pull && git checkout {branch} && git pull

# Manual Expoenent Testing
Below is an example of a manual run of TensorPrime with a custom exponent and signal (FFT) length. If no signal length is specified, the program will automatically compute the signal length as floor$($log$_2($exponent$))-2$.

In [ ]:
exponent = 4423
signal_length = 1024

!python3 main.py -p $exponent --siglen $signal_length

# PrimeNet

TensorPrime has support for receiving assignments from PrimeNet. Currently progress and result reporting are not supported.

In [ ]:
# Establish connection to PrimeNet sending over credentials
print("\nDownloading PrimeNet...\n")
# Download the primenet script from Github
user = "tdulcet"
repo = "Distributed-Computing-Scripts"
branch = "master"
pyfile = "primenet.py"

url = f"https://raw.githubusercontent.com/{user}/{repo}/{branch}/{pyfile}"
!wget --no-cache --backups=1 {url}

# Register the computer with Primenet
print("\nRegistering computer with Primenet...\n")
!python3 primenet.py -d -t 0 -W 1 -T $TPU_type_of_work -u $prime_ID -i "{'worktodo' + computer_number + '.txt'}" -r "{'results' + computer_number + '.txt'}" -l "{'local' + computer_number + '.ini'}" -H $computer_name


In [ ]:
#Get an assignment from primenet
print('\nStarting PrimeNet...\n')
!nohup python3 primenet.py -d -t 3600 -T $TPU_type_of_work -l "{'local' + computer_number + '.ini'}" >> "{'primenet' + computer_number + '.out'}" &
!sleep 1
while not os.path.exists('worktodo' + computer_number + '.txt'):
    print(f'Waiting for worktodo{computer_number}.txt access...')
    !sleep 1

## Run the cell below to kick off TensorPrime

In [ ]:
with open("/content/gdrive/MyDrive/GIMPS/TensorPrime/worktodoDefault (1).txt", "r") as filestream:
  for line in filestream:
    currentline = line.split(",")
    print("running exponent:", currentline[3])
    x = currentline[3]
    !python3 main.py -p $x